In [1]:
#%pip install -r requirements.txt

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "NA"

In [3]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3", request_timeout=4000.0)

In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

documents = SimpleDirectoryReader("data_ua").load_data()


# loads embed_model
embed_model = HuggingFaceEmbedding('benjamin/roberta-base-wechsel-ukrainian')



c:\datascience\conf_7_nov\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name benjamin/roberta-base-wechsel-ukrainian. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at benjamin/roberta-base-wechsel-ukrainian and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import chromadb
from llama_index.core import VectorStoreIndex, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext



In [6]:
# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model
# Set the chunk size (number of tokens per chunk)
Settings.chunk_size = 512
# Set the chunk overlap (number of tokens overlapping between chunks)
Settings.chunk_overlap = 50

In [7]:

index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 570/570 [05:59<00:00,  1.58it/s]


Error while downloading from https://cdn-lfs.hf.co/repos/d8/31/d83106d084f39cfb29da5f4886ac74b061803cfb0bdd23de2427b225da5d333b/b1865dda27714a19396169ffc59e4c192e0a1a519732cdc5a94c010c37519c2f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1730878644&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDg3ODY0NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9kOC8zMS9kODMxMDZkMDg0ZjM5Y2ZiMjlkYTVmNDg4NmFjNzRiMDYxODAzY2ZiMGJkZDIzZGUyNDI3YjIyNWRhNWQzMzNiL2IxODY1ZGRhMjc3MTRhMTkzOTYxNjlmZmM1OWU0YzE5MmUwYTFhNTE5NzMyY2RjNWE5NGMwMTBjMzc1MTljMmY%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=fcxmxByl2Y8dJ0zFduv-RO1f4JtHVCxCiFeTb1tSxNAw9SSnTE7eaerGr8Gj%7ER29sCMMF7oaRgFrlZ23hn8nuSly6cyEvjDjq1-5deB0kyLXiZ4yP6wldUh5ulLDOP5x47GYXx6Gq5LzSHnBQgbBoHmSofT%7EoUGTAPXHdee1EsaAyaW5HCcZonbCrEng6PY6qKQQTwYFzqPbrpB1KRf2aTl0oaAnSLCpco-0ll0IzSZs5VQ5bAOx08XHXuqWymOAVTVzngZ

In [8]:
# Set up the retriever from your index
top_k = 5
retriever = index.as_retriever(similarity_top_k=top_k)


In [9]:
def get_augmented_prompt(query):
    retrieved_documents = retriever.retrieve(query)
    # Augment the prompt by including the retrieved documents
    # Constructing an augmented prompt with context from retrieved documents
    augmented_prompt = f"Context:\n"
    for doc in retrieved_documents:
        text = doc.text
        augmented_prompt += f"{text}\n"
    augmented_prompt += f"\nQuestion: {query}\nPlease provide a concise and accurate answer in ukrainian based on the context."
    return augmented_prompt


In [10]:
from IPython.display import Markdown, display

# Send the augmented prompt to a language model for a synthesized response
def get_RAG_response(query):    
    response = llm.complete(get_augmented_prompt(query))
    return response


In [11]:
query = "Що таке інформаційне суспільство?"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))


## <b>Response:</b> 
 #### Інформаційне суспільство - це суспільство, де інформація є основною формою цінності та джерелом влади. У ньому людина живе у середовищі, що постійно генерує, оброблює та розповсюджує інформацію, що впливає на всі аспекти життя. Інформаційне суспільство характеризується швидким обміном інформацією, масовим доступом до інформації та змінами в соціальних, економічних та політичних процесах.

(What is an information society? It's a society where information is the main form of value and source of power. In it, people live in an environment that constantly generates, processes, and disseminates information that affects all aspects of life. An information society is characterized by rapid information exchange, widespread access to information, and changes in social, economic, and political processes.)

In [12]:
query = "Головні риси інформаційного суспільства?"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))

## <b>Response:</b> 
 #### Головними рисами інформаційного суспільства є:

* Розвиток комп'ютерних технологій та їхня інтеграція в різні аспекти життя;
* Центральна роль комп'ютера в обробці інформації та знання;
* Висока швидкість і глобальна доступність обміну інформацією;
* Розподіл інформації між різними джерелами та носіями;
* Використання стандартних процедур, персоналу та субкультури для організації роботи.

(Translation: The main features of an information society are:

* Development of computer technologies and their integration into various aspects of life;
* Central role of the computer in processing information and knowledge;
* High speed and global accessibility of information exchange;
* Distribution of information among different sources and carriers;
* Use of standard procedures, personnel, and subculture for organizing work.)